In [1]:
import os
import argparse
import cv2 as cv
import numpy as np

from pathlib import Path
from multiprocessing import Pool

In [6]:
# !pip install opencv-contrib-python==4.6.0.66
# !pip uninstall numpy -y
# !pip install numpy #==1.21.2

  Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)


In [2]:

# parser = argparse.ArgumentParser()
# parser.add_argument('-n', '--num_process', type=int, default=8, help='number of processes')
# parser.add_argument('--saliency-model', type=str, default='fine_grained', help='saliency model (fine_grained / spectral_residual)')
# parser.add_argument('--image-folder', type=str, help='path to image folder')
# parser.add_argument('--output-folder', type=str, help='path to save saliency map')
# parser.add_argument('--visualize-folder', type=str, default='', help='path to save saliency map visualizeualization')


circle = np.zeros((512, 512))
circle = cv.circle(circle, (256, 256), 240, 1, -1)



In [3]:

def saliency_detect(i, saliency_model, src_path, output_path, visualize_path):
    image = cv.imread(str(src_path))
    image = preprocess(image)

    if saliency_model == 'fine_grained':
        saliency = cv.saliency.StaticSaliencyFineGrained_create()
    elif saliency_model == 'spectral_residual':
        saliency = cv.saliency.StaticSaliencySpectralResidual_create()
    else:
        raise ValueError('Unknown saliency model: {}'.format(saliency_model))

    (_, raw_saliencyMap) = saliency.computeSaliency(image)
    raw_saliencyMap *= circle

    np.save(output_path, raw_saliencyMap)

    if visualize_path:
        int_saliencyMap = (raw_saliencyMap * 255).astype("uint8")
        cv.imwrite(str(visualize_path), int_saliencyMap)

    if i % 500 == 0:
        print('Processed {} images'.format(i))



def preprocess(img):
    scale = 512
    mask = np.zeros(img.shape)
    cv.circle(mask, (int(img.shape[1]/2), int(img.shape[0]/2)),
                int(scale/2*0.98), (1, 1, 1), -1, 8, 0)
    weighted_img = cv.addWeighted(img, 4, cv.GaussianBlur(img, (0, 0), scale/30), -4, 128)
    processed_img = weighted_img * mask + 128 * (1 - mask)

    # To reproduce the saliency map used in the paper,
    # we simulated the processing of saving the processed image in jpeg format and then reading it.
    # These codes can be removed if error or performance degradation is observed.
    processed_img = processed_img.astype(np.uint8)
    _, jpeg = cv.imencode('.jpeg', processed_img)
    processed_img = cv.imdecode(jpeg, cv.IMREAD_COLOR)

    return processed_img

In [8]:
# !ls ../../mnt/local/data/kalexu97
# !mkdir ../../mnt/local/data/kalexu97/visualize_test

In [4]:

def main():
    # args = parser.parse_args()
    # image_folder = Path(args.image_folder)
    # output_folder = Path(args.output_folder)
    # visualize_folder = Path(args.visualize_folder)

    image_folder = '../../mnt/local/data/kalexu97/processed_test'
    output_folder = '../../mnt/local/data/kalexu97/processed_mask_test'
    visualize_folder = '../../mnt/local/data/kalexu97/visualize_test'
    saliency_model = 'fine_grained'

    i = 0
    res = []
    pool = Pool(processes=8)
    print('Loading tasks...')
    for folder, _, imgs in os.walk(image_folder):
        folder = Path(folder)
        subfolders = folder.relative_to(image_folder)
        print(subfolders)
        # output_folder = output_folder.joinpath(subfolders)
        # output_folder.mkdir(parents=True, exist_ok=True)

        # if visualize_folder:
        #     visualize_folder = visualize_folder.joinpath(subfolders)
        #     visualize_folder.mkdir(parents=True, exist_ok=True)

        for img in imgs:
            # print(img)
            i += 1
            src_path = folder.joinpath(img)
            # output_path = output_folder.joinpath(img).with_suffix('.npy')
            output_path = output_folder + '/' + img[:-5] + '.npy'
            # visualize_path = visualize_folder.joinpath(img) if visualize_folder else ''
            visualize_path = visualize_folder + '/' + img
            res.append(pool.apply_async(saliency_detect, args=(i, saliency_model, src_path, output_path, visualize_path)))

    print('Waiting for all subprocesses done...')
    for re in res:
        re.get()
    pool.close()
    pool.join()
    print('All subprocesses done.')

main()


Loading tasks...
.
Waiting for all subprocesses done...
Processed 500 images
Processed 1000 images
Processed 1500 images
Processed 2000 images
Processed 2500 images
Processed 3000 images
Processed 3500 images
Processed 4000 images
Processed 4500 images
Processed 5000 images
Processed 5500 images
Processed 6000 images
Processed 6500 images
Processed 7000 images
Processed 7500 images
Processed 8000 images
Processed 8500 images
Processed 9000 images
Processed 9500 images
Processed 10000 images
Processed 10500 images
Processed 11000 images
Processed 11500 images
Processed 12000 images
Processed 12500 images
Processed 13000 images
Processed 13500 images
Processed 14000 images
Processed 14500 images
Processed 15000 images
Processed 15500 images
Processed 16000 images
Processed 16500 images
Processed 17000 images
Processed 17500 images
Processed 18000 images
Processed 18500 images
Processed 19000 images
Processed 19500 images
Processed 20000 images
Processed 20500 images
Processed 21000 image